In [1]:
from api_methods.get_followers import get_followers

In [2]:
MAX_DEPTH = 3
MAX_FOLLOWERS = 10

In [3]:
actor = "did:plc:lzqlhj2iss2i2athpbxnxarm"
users_to_check = set([follower.did for follower in get_followers(actor, 0)])
all_users = set(users_to_check)
curr_depth = 1

while curr_depth < MAX_DEPTH:
    next_users_to_check = set()

    for user in users_to_check:
        new_followers = set(follower.did for follower in get_followers(user, MAX_FOLLOWERS))
        next_users_to_check.update(new_followers - all_users)
        all_users.update(new_followers)

    users_to_check = next_users_to_check
    curr_depth += 1

all_followers = list(all_users)
print(all_followers[:3])

output_file = "all_followers.txt"
with open(output_file, "w") as file:
    for follower in all_followers:
        file.write(f"{follower}\n")

print(f"Saved all followers to {output_file}")

Followers retrieved: 97
Followers retrieved: 97
Followers retrieved: 97
Followers retrieved: 99
Followers retrieved: 94
Followers retrieved: 73
Total followers: 557
Followers retrieved: 80
Total followers: 80
Followers retrieved: 75
Total followers: 75
Followers retrieved: 80
Total followers: 80
Followers retrieved: 85
Total followers: 85
Followers retrieved: 40
Total followers: 40
Followers retrieved: 70
Total followers: 70
Followers retrieved: 94
Total followers: 94
Followers retrieved: 14
Total followers: 14
Followers retrieved: 56
Total followers: 56
Followers retrieved: 47
Total followers: 47
Followers retrieved: 99
Total followers: 99
Followers retrieved: 69
Total followers: 69
Followers retrieved: 6
Total followers: 6
Followers retrieved: 72
Total followers: 72
Followers retrieved: 70
Total followers: 70
Followers retrieved: 44
Total followers: 44
Followers retrieved: 94
Total followers: 94
Followers retrieved: 77
Total followers: 77
Followers retrieved: 79
Total followers: 79
F

RequestException: Response(success=False, status_code=429, content=XrpcError(error='RateLimitExceeded', message='Rate Limit Exceeded'), headers={'date': 'Mon, 02 Dec 2024 19:46:49 GMT', 'content-type': 'application/json; charset=utf-8', 'content-length': '61', 'connection': 'keep-alive', 'x-powered-by': 'Express', 'access-control-allow-origin': '*', 'ratelimit-limit': '30', 'ratelimit-remaining': '0', 'ratelimit-reset': '1733168978', 'ratelimit-policy': '30;w=300', 'etag': 'W/"3d-egyFOcXCrKQFzoI7522/4+PpxIk"', 'vary': 'Accept-Encoding'})

In [9]:
len(all_users)

4104

In [8]:
len(users_to_check)

557

In [11]:
print(all_users)

{'did:plc:ql5idw6eywus7lvuu3ewqugh', 'did:plc:245to7haebjk47ukpt5tpnsf', 'did:plc:z3bf4hu2vt3tlpi4asyltkuj', 'did:plc:66745ub6fjhkjepyikyhecyi', 'did:plc:bamdnlkxm5kq5vhrwybpqflq', 'did:plc:jchrqoaafblr5epcx2fezais', 'did:plc:lyeb6msv3k4t7xmfdolegwik', 'did:plc:whlo2tap7udbs7pgzf6oauom', 'did:plc:7ssc3vwsfylpt6pdot2zs7mw', 'did:plc:7wrkxoe2kwiidqoplffmj35q', 'did:plc:3n64ckniniw2cf7ozezo3i7w', 'did:plc:fa7yn5tewna6pl3x34j34llw', 'did:plc:kipuatspli7a5i3xdwdonw7y', 'did:plc:gchkykhql2qlghzkuzyktnlv', 'did:plc:uzurvlch4kbfuqx5maubjkyn', 'did:plc:5yvxdwtg4j7kasgg65d6hbop', 'did:plc:szu6ysqdtzrpm6yjx3nq76hs', 'did:plc:oqa24w7vkzs2qyefez6lcyza', 'did:plc:mir67bp4k6hwmp3achuhwvr5', 'did:plc:4b67tyycs42h752sgtvbbuwf', 'did:plc:tsnj2d5ebg44hndwrc5knzob', 'did:plc:hrheo2c3olfyimg3rfztryda', 'did:plc:j6euyjpqd24qiqaw2op7uyjw', 'did:plc:ek3t6uvvd2pjeudwo4j7v343', 'did:plc:2aqqfiz7ob7uetrjpqiavxzr', 'did:plc:ogrd5i7d4kp5zgksenipojwo', 'did:plc:ikw6lfso43uq673mmmeqqfio', 'did:plc:wwlt6uicsqnthhrx6t